In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df=pd.read_excel('大盤指數盤後數據.xlsx',header=0)

In [3]:
X = [] #empty list
Y = [] #empty list
for i in range(len(df.index)-10):
    X.append(list(df.loc[i:i+9]['收盤價']))
    Y.append(df.漲跌平[i+10])

In [4]:
np.any(np.isnan(df.收盤價))

False

In [5]:
X=np.array(X)
X = X.reshape(13144, 10, 1)

In [6]:
for i in range(13144):
    X[i] = X[i] - X[i].min()
    X[i] = X[i] / X[i].max()

In [24]:
%env KERAS_BACKEND = tensorflow

from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.utils import np_utils
from keras.initializers import he_normal

env: KERAS_BACKEND=tensorflow


In [8]:
Y = np_utils.to_categorical(Y, 3)

In [9]:
Y.shape

(13144, 3)

In [31]:
x = Input(shape=(10, 1))
f_1 = LSTM(20, return_sequences=True)
f_4 = Dropout(0.3)
f_2 = LSTM(20)
f_5 = Dropout(0.3)
f_3 = Dense(3, activation='softmax')

h_1 = f_1(x)
z_1 = f_4(h_1)
h_2 = f_2(z_1)
z_2 = f_5(h_2)
y = f_3(z_2)

model = Model(x, y)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 10, 1)             0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 10, 20)            1760      
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 20)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 20)                3280      
_________________________________________________________________
dropout_6 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 63        
Total params: 5,103
Trainable params: 5,103
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(loss="categorical_crossentropy", optimizer='RMSprop', metrics=['accuracy'])

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state = 9487)

In [35]:
model.fit(x_train, y_train, epochs=50)

Epoch 1/50
9200/9200 [==============================] - 10s 1ms/step - loss: 1.0954 - acc: 0.3683
Epoch 2/50
9200/9200 [==============================] - 6s 674us/step - loss: 1.0935 - acc: 0.3687
Epoch 3/50
9200/9200 [==============================] - 6s 671us/step - loss: 1.0925 - acc: 0.3655
Epoch 4/50
9200/9200 [==============================] - 6s 678us/step - loss: 1.0923 - acc: 0.3652
Epoch 5/50
9200/9200 [==============================] - 6s 676us/step - loss: 1.0920 - acc: 0.3678
Epoch 6/50
9200/9200 [==============================] - 6s 671us/step - loss: 1.0918 - acc: 0.3674 0s - loss: 1.0917 -
Epoch 7/50
9200/9200 [==============================] - 6s 677us/step - loss: 1.0912 - acc: 0.3730 0s - loss: 1.0910 - a
Epoch 8/50
9200/9200 [==============================] - 6s 674us/step - loss: 1.0921 - acc: 0.3709
Epoch 9/50
9200/9200 [==============================] - 6s 671us/step - loss: 1.0913 - acc: 0.3711
Epoch 10/50
9200/9200 [==============================] - 6s 673us/st

In [36]:
model.evaluate(x_test, y_test)

3944/3944 [==============================] - 1s 174us/step


[1.0877780174386913, 0.38336713995943206]

In [37]:
model.predict(x_test)

array([[0.30335853, 0.33117032, 0.36547115],
       [0.34851152, 0.4745044 , 0.17698404],
       [0.34723076, 0.33846673, 0.31430247],
       ...,
       [0.3252829 , 0.3160709 , 0.35864618],
       [0.33170927, 0.3453784 , 0.3229123 ],
       [0.32153714, 0.3211075 , 0.35735542]], dtype=float32)

In [38]:
predict = model.predict(x_test)
np.argmax(predict,axis=1)

array([2, 1, 0, ..., 2, 1, 2], dtype=int64)

In [44]:
y_test

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)

In [39]:
from sklearn.metrics import confusion_matrix

In [45]:
confusion_matrix(np.argmax(y_test,axis=1), np.argmax(predict,axis=1))

array([[760, 402, 350],
       [583, 354, 314],
       [486, 297, 398]], dtype=int64)

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(20, 20))

plt.plot(df.收盤價, '-')
plt.plot(model.predict(X), '-')

plt.show()